In [1]:
import sys
print(sys.executable)

c:\Users\ADMIN\Desktop\MY PROJECT\Deployed projects\post_surgery_complication\venv\Scripts\python.exe


In [2]:
import os

print(os.getcwd())

c:\Users\ADMIN\Desktop\MY PROJECT\Deployed projects\post_surgery_complication\notebooks


In [3]:
import pandas as pd
df = pd.read_csv("../data/raw/post-operative-data.csv")
df.head()

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision ADM-DECS
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   L-CORE             90 non-null     object
 1   L-SURF             90 non-null     object
 2   L-O2               90 non-null     object
 3   L-BP               90 non-null     object
 4   SURF-STBL          90 non-null     object
 5   CORE-STBL          90 non-null     object
 6   BP-STBL            90 non-null     object
 7   COMFORT            90 non-null     object
 8   decision ADM-DECS  90 non-null     object
dtypes: object(9)
memory usage: 6.5+ KB


In [5]:
df.shape

(90, 9)

In [6]:
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

L-CORE: 3 unique values
L-SURF: 3 unique values
L-O2: 2 unique values
L-BP: 3 unique values
SURF-STBL: 2 unique values
CORE-STBL: 3 unique values
BP-STBL: 3 unique values
COMFORT: 5 unique values
decision ADM-DECS: 4 unique values


In [7]:
for col in df.columns:
    print(col,'->',df[col].unique())

L-CORE -> ['mid' 'high' 'low']
L-SURF -> ['low' 'high' 'mid']
L-O2 -> ['excellent' 'good']
L-BP -> ['mid' 'high' 'low']
SURF-STBL -> ['stable' 'unstable']
CORE-STBL -> ['stable' 'unstable' 'mod-stable']
BP-STBL -> ['stable' 'mod-stable' 'unstable']
COMFORT -> ['15' '10' '5' '7' '?']
decision ADM-DECS -> ['A' 'S' 'A ' 'I']


In [8]:
df['decision ADM-DECS'].value_counts()

decision ADM-DECS
A     63
S     24
I      2
A      1
Name: count, dtype: int64

In [9]:
df.isnull().sum()

L-CORE               0
L-SURF               0
L-O2                 0
L-BP                 0
SURF-STBL            0
CORE-STBL            0
BP-STBL              0
COMFORT              0
decision ADM-DECS    0
dtype: int64

In [10]:
df.columns=df.columns.str.strip().str.replace(' ','_').str.replace('-','_')
df.head()

,L_CORE,L_SURF,L_O2,L_BP,SURF_STBL,CORE_STBL,BP_STBL,COMFORT,decision_ADM_DECS
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [11]:
import numpy as np

df.replace("?", np.nan, inplace=True)

# Convert COMFORT to numeric
df["COMFORT"] = pd.to_numeric(df["COMFORT"])

In [12]:
df["decision_ADM_DECS"] = df["decision_ADM_DECS"].str.strip()

In [13]:
df.isnull().sum()
df["decision_ADM_DECS"].value_counts()

decision_ADM_DECS
A    64
S    24
I     2
Name: count, dtype: int64

In [14]:
# Converting labels to binary values
df['decision_ADM_DECS']=df["decision_ADM_DECS"].replace({
    "A": 0,   # Safe to discharge
    "S": 1,   # Requires monitoring / potential complication
    "I": 1    # Requires monitoring (rare but critical)
})

print('updated class distribution:')
print(df['decision_ADM_DECS'].value_counts())

updated class distribution:
decision_ADM_DECS
0    64
1    26
Name: count, dtype: int64


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14708\3995801245.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['decision_ADM_DECS']=df["decision_ADM_DECS"].replace({


In [15]:
X = df.drop('decision_ADM_DECS', axis=1)
y = df['decision_ADM_DECS']

In [16]:
X.dtypes

L_CORE        object
L_SURF        object
L_O2          object
L_BP          object
SURF_STBL     object
CORE_STBL     object
BP_STBL       object
COMFORT      float64
dtype: object

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# All columns are categorical, so no numeric columns
categorical_cols = X.columns.tolist()

# Encoder setup
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), categorical_cols)
    ],
    remainder="passthrough"
)

# Split data before fitting (important)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

Train size: (72, 8)
Test size: (18, 8)


In [24]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [25]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

rf_smote_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("smote", smote),
    ("model", RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        random_state=42
    ))
])

rf_smote_clf.fit(X_train, y_train)

y_pred_smote = rf_smote_clf.predict(X_test)
y_proba_smote = rf_smote_clf.predict_proba(X_test)[:, 1]

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

log_reg_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

In [28]:
log_reg_clf.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [29]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

y_pred_lr = log_reg_clf.predict(X_test)
y_proba_lr = log_reg_clf.predict_proba(X_test)[:, 1]

print("Confusion Matrix (LogReg):")
print(confusion_matrix(y_test, y_pred_lr))

print("\nClassification Report (LogReg):")
print(classification_report(y_test, y_pred_lr))

print("ROC-AUC (LogReg):", roc_auc_score(y_test, y_proba_lr))

Confusion Matrix (LogReg):
[[6 7]
 [3 2]]

Classification Report (LogReg):
              precision    recall  f1-score   support

           0       0.67      0.46      0.55        13
           1       0.22      0.40      0.29         5

    accuracy                           0.44        18
   macro avg       0.44      0.43      0.42        18
weighted avg       0.54      0.44      0.47        18

ROC-AUC (LogReg): 0.3384615384615385


In [30]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced"
    ))
])

In [31]:
rf_clf.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [32]:
y_pred_rf = rf_clf.predict(X_test)
y_proba_rf = rf_clf.predict_proba(X_test)[:, 1]

print("Confusion Matrix (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf))

print("ROC-AUC (Random Forest):", roc_auc_score(y_test, y_proba_rf))

Confusion Matrix (Random Forest):
[[12  1]
 [ 5  0]]

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.71      0.92      0.80        13
           1       0.00      0.00      0.00         5

    accuracy                           0.67        18
   macro avg       0.35      0.46      0.40        18
weighted avg       0.51      0.67      0.58        18

ROC-AUC (Random Forest): 0.5076923076923077


In [33]:
print(confusion_matrix(y_test, y_pred_smote))
print(classification_report(y_test, y_pred_smote))
print("ROC-AUC (RF + SMOTE):", roc_auc_score(y_test, y_proba_smote))

[[11  2]
 [ 4  1]]
              precision    recall  f1-score   support

           0       0.73      0.85      0.79        13
           1       0.33      0.20      0.25         5

    accuracy                           0.67        18
   macro avg       0.53      0.52      0.52        18
weighted avg       0.62      0.67      0.64        18

ROC-AUC (RF + SMOTE): 0.5230769230769231


In [34]:
pip install catboost


   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.8/102.4 MB 8.5 MB/s eta 0:00:13
    --------------------------------------- 2.4/102.4 MB 7.9 MB/s eta 0:00:13
   - -------------------------------------- 3.9/102.4 MB 7.6 MB/s eta 0:00:13
   -- ------------------------------------- 5.5/102.4 MB 7.5 MB/s eta 0:00:13
   -- ------------------------------------- 7.1/102.4 MB 7.4 MB/s eta 0:00:13
   --- ------------------------------------ 8.7/102.4 MB 7.4 MB/s eta 0:00:13
   --- ------------------------------------ 10.2/102.4 MB 7.3 MB/s eta 0:00:13
   ---- ----------------------------------- 11.8/102.4 MB 7.3 MB/s eta 0:00:13
   ----- ---------------------------------- 13.1/102.4 MB 7.3 MB/s eta 0:00:13
   ----- ---------------------------------- 14.7/102.4 MB 7.3 MB/s eta 0:00:13
   ------ --------------------------------- 16.3/102.4 MB 7.2 MB/s eta 0:00:12
   ------ --------------------------------- 17.8/102.4 MB 7.2 MB/s

In [36]:
X.dtypes

L_CORE        object
L_SURF        object
L_O2          object
L_BP          object
SURF_STBL     object
CORE_STBL     object
BP_STBL       object
COMFORT      float64
dtype: object

In [37]:
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
categorical_cols

['L_CORE', 'L_SURF', 'L_O2', 'L_BP', 'SURF_STBL', 'CORE_STBL', 'BP_STBL']

In [38]:
cat_feature_indices = [X.columns.get_loc(col) for col in categorical_cols]
cat_feature_indices

[0, 1, 2, 3, 4, 5, 6]

In [39]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Step 1: build cat feature index list
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
cat_feature_indices = [X.columns.get_loc(col) for col in categorical_cols]

cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_state=42,
    class_weights=[1, 3]  # more weight for class 1 (complication)
)

cat_model.fit(
    X_train, 
    y_train, 
    cat_features=cat_feature_indices, 
    verbose=False
)

y_pred_cat = cat_model.predict(X_test)
y_proba_cat = cat_model.predict_proba(X_test)[:, 1]

print("Confusion Matrix (CatBoost):")
print(confusion_matrix(y_test, y_pred_cat))

print("\nClassification Report (CatBoost):")
print(classification_report(y_test, y_pred_cat))

print("ROC-AUC (CatBoost):", roc_auc_score(y_test, y_proba_cat))

Confusion Matrix (CatBoost):
[[13  0]
 [ 5  0]]

Classification Report (CatBoost):
              precision    recall  f1-score   support

           0       0.72      1.00      0.84        13
           1       0.00      0.00      0.00         5

    accuracy                           0.72        18
   macro avg       0.36      0.50      0.42        18
weighted avg       0.52      0.72      0.61        18

ROC-AUC (CatBoost): 0.5692307692307692


c:\Users\ADMIN\Desktop\MY PROJECT\Deployed projects\post_surgery_complication\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ADMIN\Desktop\MY PROJECT\Deployed projects\post_surgery_complication\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ADMIN\Desktop\MY PROJECT\Deployed projects\post_surgery_complication\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predic

In [40]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Get probabilities instead of hard class labels
y_proba_lr = log_reg_clf.predict_proba(X_test)[:, 1]

# Use a lower threshold for class 1 (complication)
threshold = 0.3
y_pred_lr_tuned = (y_proba_lr >= threshold).astype(int)

print("Confusion Matrix (LogReg, threshold=0.3):")
print(confusion_matrix(y_test, y_pred_lr_tuned))

print("\nClassification Report (LogReg, threshold=0.3):")
print(classification_report(y_test, y_pred_lr_tuned))

print("ROC-AUC (LogReg):", roc_auc_score(y_test, y_proba_lr))


Confusion Matrix (LogReg, threshold=0.3):
[[ 2 11]
 [ 2  3]]

Classification Report (LogReg, threshold=0.3):
              precision    recall  f1-score   support

           0       0.50      0.15      0.24        13
           1       0.21      0.60      0.32         5

    accuracy                           0.28        18
   macro avg       0.36      0.38      0.28        18
weighted avg       0.42      0.28      0.26        18

ROC-AUC (LogReg): 0.3384615384615385
